<a href="https://colab.research.google.com/github/wayamhui/ISYS5002_Assignment/blob/main/Assignment20486978.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment20486978.ipynb

In [1]:
# install pip and import useful modules

!pip install kora -q
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd



/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import pandas as pd

In [3]:
# Tax calculation function standby in here, and it is used in the add_tax_in_records function

def tax_cal(annualincome):
  if annualincome <=18200:
    tax_payable = 0
  elif annualincome <=45000:
    tax_payable = ( annualincome - 18200 ) * 0.19 
  elif annualincome <=120000:
    tax_payable = ( annualincome - 45000 ) * 0.325 + 5092
  elif annualincome <=180000:
    tax_payable = ( annualincome - 120000 ) * 0.37 + 29467
  else:
    tax_payable = ( annualincome - 180000 ) * 0.45 + 51667
  return tax_payable


In [4]:
# URL creation function ready

# user input ticker symbol


def build_url(ticker_symbol):

  # print(ticker_symbol)
  # user tikcer to get profile page
  example_url = 'https://finance.yahoo.com/quote/{}/profile?p={}'
  result = example_url.format (ticker_symbol,ticker_symbol)
  return result

In [15]:
# Validate the input for ticker

def validate_url_of_ticker(ticker_symbol):   # url as an input
  valid = False   # F needs to be capitalized

  while not valid:
    try:      
      # print(url)
      url = build_url(ticker_symbol)
      wd.get(url)    

      soup = BeautifulSoup(wd.page_source, "html.parser")

      table = soup.find('table')
      body = soup.find('tbody')
      rows = body.find_all('tr')
      
      valid = True

    except AttributeError:
      print("The ticker inputted seems not a valid company, please re-input ticker symbol for a comany...") 
      ticker_symbol = input ("Please input a ticker symbol to view the excecutives' annual pay and the calculated AXO tax (e.g. NAB.AX): ")
  return ticker_symbol


# urltest = validate_url_of_ticker('nab.ax')


In [6]:
def get_rows_from_url_table(url):  # url as an input

  wd.get(url)
  soup = BeautifulSoup(wd.page_source, "html.parser")
  table = soup.find('table')
  body = soup.find_all('tbody')

  # print (body)
  # print (len(body))

  body = soup.find('tbody')

  # print (body)
  # print (len(body))

  rows = body.find_all('tr')
  # print (len(rows))

  row1 = rows[0]
  cells = row1.find_all('td')
  # print (len(cells))

  name = cells[0]
  title = cells[1]
  pay = cells[2]

  return rows

# rows = get_rows_from_url_table()

def get3cells(row):    
  cells = row.find_all('td')
  # print (len(cells))
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  
  # for item in [0,1,2,3,4]:
  #  print (cells[item].text)  # to test print 4 cells together.

  return {'name': name, 'title': title, 'pay': pay}  # using dictionary type, so no need to remember zero refers to name, 1 refers to title, etc)

# print (get3cells(rows[3])) # hange to commentary, after tested

In [7]:
# get executive pay from the URL's profile table to construct records in our dictionary

def get_exec_records(rows):  # rows is used as an input to this function
  exec_records = []
  for row in rows:
    result = get3cells(row)
    exec_records.append(result)
  return exec_records

# print (get_exec_records({xxxxx}))

In [8]:
# validate foreign exchange rate

def validate_fx(fx):
  valid = False
  while not valid:
    try:       
      if fx == "":
        fx = 1.0    
      else:
        fx = float(fx)
      valid = True
    except ValueError:
      print ('Please re-input FX in number format (decimal points are accepted)...')
      fx = input('If the company of the ticker is not listed in Australia, please input FX: 1 ForeignCurrency = ? AUD (Enter blank if Australia listed): ')
  return fx

# validate_fx('a')
# validate_fx(2)

In [9]:
def thousand_translate(pay_display): # this is to translate the pay in "M" and "k" format to normal number format
  last_char = pay_display[-1]
  last3_char = pay_display[-3:-1] + pay_display[-1] 

  if last3_char == "N/A":
    salary = -1     # to distignuish N/A (as -1) from zero salary
    
  elif last_char == 'M':    
    number = pay_display[0:-1]
    salary = float(number) * 1000000
  elif last_char == 'k': 
    number = pay_display[0:-1]
    salary = float(number) * 1000  
  else:
    salary = float(pay_display)  # some executives can have salary of $1 (i.e. not ending with k or M)
  return salary


# print (thousand_translate('3k'))
# records = get_exec_records()

In [10]:
def add_tax_in_records(records):
  for record in records:
    income = record['pay']
    
    temp = thousand_translate(income)
    
    if temp == -1:
      record['tax'] = 'N/A'
    
    else:
      temp1 = tax_cal(temp * fxtoaud)   # global fx, ,not local variable
      if temp1 > 1000000:         # this if phrase is to convert big amounts to "M" or "k" format with 2 decimal point that is consistnent with pay
        temp2 = str(round(temp1/1000000, 2)) +"M"
      elif temp1 > 1000:
        temp2 = str(round(temp1/1000, 2)) +"k"
      else:
        temp2 = str(temp1)
      record['tax'] = temp2
      # print (record['tax'])  # test print before finalizing the function
  return records  
    
# add_tax_in_records(validate_fx())

In [31]:
def output_to_cvs(name):
  
  fullname = '/content/' + name + '.csv'  
  with open(fullname,'w') as f:
    f.writelines("Name \t Title \t Pay(Original_Currency) \t Tax(AUD)")
    f.writelines("\n")
    for record in records:
      temp3 = str(record['name'])+ "\t" + str(record['title']) + "\t" + str(record['pay']) + "\t" + str(record['tax']) 
      f.writelines(temp3)
      f.writelines("\n")  
  f.close()

output_to_cvs('tab')

In [64]:
def view_records():
  print ("Pay(OriginalCurrency) \tTax(AUD) \tName & Title")
    
  for record in records:
      temp4 = str(record['pay']) + "\t"+"\t"+"\t" + str(record['tax'] + "\t"+"\t" + str(record['name'])+ " as " + str(record['title'])) 
      print(temp4)
  


# This session is the main program by recalling the functions defined in above. 
This session can be run multiple times, all previous session only need to run once. 

In [32]:
# main input and validation:

ticker = input ("Please input a ticker symbol to view the excecutives' annual pay and the calculated AXO tax (e.g. NAB.AX): ")
updatedticker = validate_url_of_ticker(ticker)
url = build_url(updatedticker)

fx = input('If the company of the ticker is not listed in Australia, please input FX: 1 ForeignCurrency = ? AUD: (Enter blank if Australia listed) ')
fxtoaud = validate_fx(fx)

  # print (url)

# functions used:
  
rows = get_rows_from_url_table(url)

records = get_exec_records(rows)

  # print (records)

add_tax_in_records(records)

  # print (records) 

#  Ask user to choose view records immediately or to save to csv file

action = input ("Would you like to VIEW the executives' tax of ticker '"+ updatedticker + "' now or SAVE it to cvs file, or BOTH view and save? (V/S/B)")

valid = False

while not valid:
  if action in ['V','S','B','v','s','b']: 
    valid = True
  else:
    print ('Please re-input, V for View, S for Save, and B for Both')
    action = input ("Would you like to VIEW the executives' tax of ticker '"+ updatedticker + "' now or SAVE it to cvs file, or BOTH view and save? (V/S/B)")  
    
if action in ['V','B','v','b']: 
      print("\n")
      view_records()  
      print("Thanks for viewing")
      print("\n")
if action in ['S','B','s','b']:
      filename = input('Please input the filename you would like to save as a csv file (Enter blank to use the first 3 charactors of the ticker as filename): ')
      if filename == '':
        output_to_cvs(updatedticker[0:3])  # use first 3 charactors to avoid picking the dot for use in filename
        filename = updatedticker[0:3]
      else:
        output_to_cvs(filename)
      print("The file " + filename + ".cvs has been saved. Thanks.") 




Please input a ticker symbol to view the excecutives' annual pay and the calculated AXO tax (e.g. NAB.AX): goog
If the company of the ticker is not listed in Australia, please input FX: 1 ForeignCurrency = ? AUD: (Enter blank if Australia listed) 1
https://finance.yahoo.com/quote/goog/profile?p=goog
[{'name': 'Mr. Sundar  Pichai', 'title': 'CEO & Director', 'pay': '7.43M', 'tax': '3.31M'}, {'name': 'Mr. Lawrence Edward Page', 'title': 'Co-Founder & Director', 'pay': '1', 'tax': '0'}, {'name': 'Mr. Sergey  Brin', 'title': 'Co-Founder & Director', 'pay': '1', 'tax': '0'}, {'name': 'Ms. Ruth M. Porat', 'title': 'Sr. VP & CFO', 'pay': '672.77k', 'tax': '273.41k'}, {'name': 'Dr. Prabhakar  Raghavan', 'title': 'Sr. VP of Google', 'pay': '664.75k', 'tax': '269.8k'}, {'name': 'Mr. Philipp  Schindler', 'title': 'Sr. VP & Chief Bus. Officer of Google', 'pay': '881.82k', 'tax': '367.49k'}, {'name': 'Mr. Kent  Walker', 'title': 'Corp. Sec.', 'pay': '664.75k', 'tax': '269.8k'}, {'name': "Ms. Amie T

Would you like to VIEW the executives' tax of ticker 'goog' now or SAVE it to cvs file, or BOTH view and save? (V/S/B)t
Please re-input, V for View, S for Save, and B for Both
Would you like to VIEW the executives' tax of ticker 'goog' now or SAVE it to cvs file, or BOTH view and save? (V/S/B)
Please re-input, V for View, S for Save, and B for Both
Would you like to VIEW the executives' tax of ticker 'goog' now or SAVE it to cvs file, or BOTH view and save? (V/S/B)
Please re-input, V for View, S for Save, and B for Both
Would you like to VIEW the executives' tax of ticker 'goog' now or SAVE it to cvs file, or BOTH view and save? (V/S/B)t
Please re-input, V for View, S for Save, and B for Both
Would you like to VIEW the executives' tax of ticker 'goog' now or SAVE it to cvs file, or BOTH view and save? (V/S/B)t
Please re-input, V for View, S for Save, and B for Both
Would you like to VIEW the executives' tax of ticker 'goog' now or SAVE it to cvs file, or BOTH view and save? (V/S/B)b


